In [1]:
import yfinance as yf
import datetime
import ta
import datetime
from ta import momentum, trend

symbol = "^NSEI"
ticker = yf.Ticker(symbol)
end_date = datetime.date.today()
df = ticker.history(period="10d", interval="5m")
df['Date'] = df.index.date
df['Time'] = df.index.time

In [2]:
# dir(ticker)

In [3]:
# ticker.info
# ticker.news

In [4]:

from ta import momentum,trend
df['EMA'] = trend.EMAIndicator(df['Close'], window=50).ema_indicator()
df['RSI'] = round(momentum.RSIIndicator(df['Close'], window=4).rsi(),2)

df = df.dropna()
df = df[df['RSI'] != 0]

In [5]:
# df['Previous_5_RSI'] = [df['RSI'][max(0, i-5):i].to_list() if i >= 5 else [] for i in range(len(df))]
df['RSI_DATA'] = [df['RSI'][max(0, i-1):i+1].to_list() if i >= 4 else [] for i in range(len(df))]

In [6]:
def check_order(arr):
    increasing = all(arr[i] <= arr[i + 1] for i in range(len(arr) - 1))
    decreasing = all(arr[i] >= arr[i + 1] for i in range(len(arr) - 1))
    return "Increasing" if increasing else "Decreasing" if decreasing else "None"


In [7]:
df['RSI_Status'] = df['RSI_DATA'].apply(check_order)

In [8]:
# df[df['RSI_Status'] == 'Decreasing']
# df.sort_values(by="Datetime", ascending=False, inplace=True)
# df

In [9]:
# df.head(20)

In [10]:
inisal_amount = 30000
position = False
def PLSTATUS(row):
    global inisal_amount
    global position
    lot_size = 50
    current_price = row['Close'] * lot_size
    
    # if row['RSI'] <= 30:
    if row['RSI'] <= 30 and row['RSI_Status'] in ["Increasing", "None"]:
        if position == False:
            print(f"------[{row['Date']}]----------------PE EXTRY --[{row['Time']}]---------------RSI [{row['RSI']}]--------------{row['Close']}-----------")
            inisal_amount = inisal_amount - current_price
            print(inisal_amount)
            position = True
            
    # elif row['RSI'] >= 70:
    elif row['RSI'] >= 70 and row['RSI_Status'] == "Decreasing":
        if position == True:
            print(f"-------[{row['Date']}]----------------PE EXIT --[{row['Time']}]-------------RSI [{row['RSI']}]-------------{row['Close']}------------")
            inisal_amount = inisal_amount + current_price
            print(inisal_amount)
            position = False
    
   

In [11]:
df.apply(PLSTATUS,axis = 1)

------[2023-10-11]----------------PE EXTRY --[11:15:00]---------------RSI [24.35]--------------19808.900390625-----------
-960445.01953125
-------[2023-10-11]----------------PE EXIT --[14:55:00]-------------RSI [70.22]-------------19823.099609375------------
30709.9609375
------[2023-10-12]----------------PE EXTRY --[09:50:00]---------------RSI [26.54]--------------19793.30078125-----------
-958955.078125
-------[2023-10-13]----------------PE EXIT --[11:30:00]-------------RSI [75.68]-------------19692.44921875------------
25667.3828125
------[2023-10-16]----------------PE EXTRY --[09:20:00]---------------RSI [22.1]--------------19714.0-----------
-960032.6171875
-------[2023-10-16]----------------PE EXIT --[10:10:00]-------------RSI [78.95]-------------19749.30078125------------
27432.421875
------[2023-10-16]----------------PE EXTRY --[13:20:00]---------------RSI [23.13]--------------19748.55078125-----------
-959995.1171875
-------[2023-10-17]----------------PE EXIT --[09:25:00]-----

Datetime
2023-10-10 13:20:00+05:30    None
2023-10-10 13:25:00+05:30    None
2023-10-10 13:30:00+05:30    None
2023-10-10 13:35:00+05:30    None
2023-10-10 13:40:00+05:30    None
                             ... 
2023-10-23 15:05:00+05:30    None
2023-10-23 15:10:00+05:30    None
2023-10-23 15:15:00+05:30    None
2023-10-23 15:20:00+05:30    None
2023-10-23 15:25:00+05:30    None
Length: 701, dtype: object

In [12]:
if position == True :
    print("HOLDING")
    last_price_today = round(df.iloc[1]['Close'],2)*50
    print("MY AMOUNT : 30000 ")
    print("TOTAL RETURNS  : ",last_price_today + inisal_amount)
    print("MY P&L : ",last_price_today + inisal_amount-30000)
    percentage = (100 * (last_price_today + inisal_amount-30000))/30000 
    print(f"MY P& %: {percentage}%")
    
else:
    print("MY AMOUNT : 30000 ")
    print("TOTAL RETURNS  : ",inisal_amount)
    print("MY P&L : ",inisal_amount-30000)
    percentage = (100 * (inisal_amount-30000))/30000 
    print(f"MY P&L %: {percentage}%")

HOLDING
MY AMOUNT : 30000 
TOTAL RETURNS  :  36477.32421875012
MY P&L :  6477.324218750116
MY P& %: 21.591080729167054%


In [13]:
# MY AMOUNT : 30000 
# TOTAL RETURNS  :  32725.0
# MY P&L :  2725.0
# MY P& %: 9.083333333333334%
#  5 minutes 1 day